In [15]:
import sys

from models.DeepDense import DeepDense
from models.TextLSTM import TextLSTM
from models.DNNAttr import DNNAttr
from models.Wide import Wide

from models.WideDeep import WideDeep
from optim.Initializer import KaimingNormal, XavierNormal
from optim.radam import RAdam
from pandas import DataFrame
from preprocessing.Preprocessor import WidePreprocessor, DeepPreprocessor, DeepTextPreprocessor, MultiDeepTextPreprocessor

import numpy as np
import pandas as pd
import torch, os
from torch.utils.data import DataLoader
import time
import torch


wide = Wide(wide_dim=17, output_dim=1)

# init deep_dense model
deep_column_idx = dict()
deep_column_idx['family_pred_gender'] = 0
deep_column_idx['family_pred_age_level'] = 1
deep_column_idx['category'] = 2
deep_column_idx['ott_uv_norm'] = 3
deep_column_idx['category_prefer'] = 4
emb_col_val_dim_tuple = []
emb_col_val_dim_tuple.append(('family_pred_gender', 4, 8))
emb_col_val_dim_tuple.append(('family_pred_age_level', 1024, 12))
emb_col_val_dim_tuple.append(('category', 28, 8))
deepdense = DeepDense(hidden_layers=[32], dropout=[0.2], deep_column_idx=deep_column_idx, embed_input=emb_col_val_dim_tuple, continuous_cols=['ott_uv_norm', 'category_prefer'])

# init transformer model
transformer = DNNAttr()

wide_deep_model = WideDeep(wide=wide, deepdense=deepdense, deeptext=transformer, head_layers=[128])
wide_deep_model.load_state_dict(torch.load('log/05-06_16.29/sug_saved_model.pt'))

for name, param in wide_deep_model.named_parameters():
    print(name, '        ', param.size())

deep dense 维度：32
wide.wide_linear.weight          torch.Size([1, 17])
wide.wide_linear.bias          torch.Size([1])
deepdense.embed_layers_dic.emb_layer_category.weight          torch.Size([28, 8])
deepdense.embed_layers_dic.emb_layer_family_pred_age_level.weight          torch.Size([1024, 12])
deepdense.embed_layers_dic.emb_layer_family_pred_gender.weight          torch.Size([4, 8])
deepdense.dense_sequential.dense_layer_0.0.weight          torch.Size([32, 30])
deepdense.dense_sequential.dense_layer_0.0.bias          torch.Size([32])
deeptext.embedding.weight          torch.Size([180000, 32])
deeptext.fc.weight          torch.Size([64, 32])
deeptext.fc.bias          torch.Size([64])
prefix_embedding.weight          torch.Size([105000, 32])
deephead.head_layer_0.0.weight          torch.Size([128, 160])
deephead.head_layer_0.0.bias          torch.Size([128])
deephead.head_out.weight          torch.Size([1, 128])
deephead.head_out.bias          torch.Size([1])


In [ ]:
wide_linear_weight=''
wide_linear_bias=''
emb_layer_category=''
emb_layer_family_pred_age_level=''
emb_layer_family_pred_gender=''
dense_layer_0_w=''
dense_layer_0_bias=''
dense_layer_1_w=''
dense_layer_1_bias=''
query_embedding=''
query_embedding_fc_w=''
query_embedding_fc_bias=''
prefix_embedding=''
head_layer_0_w=''
head_layer_0_bias=''
head_layer_1_w=''
head_layer_1_bias=''
head_out_w=''
head_out_bias=''
for name, param in wide_deep_model.named_parameters():
    
    if 'wide.wide_linear.weight' in name:
        wide_linear_weight = np.round(param.detach().numpy(), 3)
    
    if 'wide.wide_linear.bias' in name:
        wide_linear_bias = np.round(param.detach().numpy(), 3)
    
    # deepdense
    if 'deepdense.embed_layers_dic.emb_layer_category.weight' in name:
        emb_layer_category = np.round(param.detach().numpy(), 3)
    
    if 'deepdense.embed_layers_dic.emb_layer_family_pred_age_level.weight' in name:
        emb_layer_family_pred_age_level = np.round(param.detach().numpy(), 3)
        
    if 'deepdense.embed_layers_dic.emb_layer_family_pred_gender.weight' in name:
        emb_layer_family_pred_gender = np.round(param.detach().numpy(), 3)
    
    if 'deepdense.dense_sequential.dense_layer_0.0.weight' in name:
        dense_layer_0_w = np.round(param.detach().numpy(), 3)
    if 'deepdense.dense_sequential.dense_layer_0.0.bias' in name:
        dense_layer_0_bias = np.round(param.detach().numpy(), 3)
    if 'deepdense.dense_sequential.dense_layer_1.0.weight' in name:
        dense_layer_1_w = np.round(param.detach().numpy(), 3)
    if 'deepdense.dense_sequential.dense_layer_1.0.bias' in name:
        dense_layer_1_bias = np.round(param.detach().numpy(), 3)
    
    
    # text
    if 'deeptext.embedding.weight' in name:
        query_embedding = np.round(param.detach().numpy(), 3)
    if 'deeptext.fc.weight' in name:
        query_embedding_fc_w = np.round(param.detach().numpy(), 3)
    if 'deeptext.fc.bias' in name:
        query_embedding_fc_bias = np.round(param.detach().numpy(), 3)
    
    # prefix
    if 'prefix_embedding.weight' in name:
        prefix_embedding = np.round(param.detach().numpy(), 3)
    if 'deephead.head_layer_0.0.weight' in name:
        head_layer_0_w = np.round(param.detach().numpy(), 3)
    if 'deephead.head_layer_0.0.bias' in name:
        head_layer_0_bias = np.round(param.detach().numpy(), 3)
    
    if 'deephead.head_layer_1.0.weight' in name:
        head_layer_1_w = np.round(param.detach().numpy(), 3)
    if 'deephead.head_layer_1.0.bias' in name:
        head_layer_1_bias = np.round(param.detach().numpy(), 3)
    
    if 'deephead.head_out.weight' in name:
        head_out_w = np.round(param.detach().numpy(), 3)
    
    if 'deephead.head_out.bias' in name:
        head_out_bias = np.round(param.detach().numpy(), 3)
    
np.savez("param.npz", wide_linear_weight=wide_linear_weight,
                        wide_linear_bias=wide_linear_bias,
                        emb_layer_category=emb_layer_category,
                        emb_layer_family_pred_age_level=emb_layer_family_pred_age_level,
                        emb_layer_family_pred_gender=emb_layer_family_pred_gender,
                        dense_layer_0_w=dense_layer_0_w,
                        dense_layer_0_bias=dense_layer_0_bias,
                        dense_layer_1_w=dense_layer_1_w,
                        dense_layer_1_bias=dense_layer_1_bias,
                        query_embedding=query_embedding,
                        query_embedding_fc_w=query_embedding_fc_w,
                        query_embedding_fc_bias=query_embedding_fc_bias,
                        prefix_embedding=prefix_embedding,
                        head_layer_0_w=head_layer_0_w,
                        head_layer_0_bias=head_layer_0_bias,
                        head_layer_1_w=head_layer_1_w,
                        head_layer_1_bias=head_layer_1_bias,
                        head_out_w=head_out_w,
                        head_out_bias=head_out_bias)
    
    
#     if 'emb_layer_family_pred_age_level' in name:
#         emb_layer_family_pred_age_level = np.round(param.detach().numpy(), 3)
        
#         print(name)
#         print(emb_layer_family_pred_age_level)

#     if 'wide.wide_linear.weight' in name:
#         print(wide_linear_weight)
        
        
np_file = np.load("param.npz")
print(np_file['query_embedding'].shape)
print(np_file['query_embedding'])




In [22]:
# [rows, cols] = np_file['dense_layer_1_bias'].shape

print(np_file['dense_layer_0_w'])


[[ 0.037 -0.07   0.191 -0.085 -0.075 -0.149  0.01   0.123  0.152  0.067
  -0.24  -0.146  0.18  -0.264 -0.054  0.262 -0.257 -0.032 -0.061 -0.056
   0.178  0.311 -0.277 -0.124 -0.401  0.112  0.153  0.029 -0.213 -0.218]
 [ 0.003  0.2    0.071  0.149  0.133 -0.142 -0.322  0.204 -0.238  0.297
  -0.024  0.067 -0.03  -0.155  0.163  0.094 -0.136 -0.326  0.048  0.09
   0.164  0.012 -0.128  0.277  0.121  0.152  0.004  0.126  0.154  0.144]
 [ 0.298  0.151 -0.082 -0.084  0.054 -0.289  0.225  0.129  0.004  0.089
  -0.194  0.217  0.04   0.198  0.078 -0.079  0.026 -0.135 -0.136 -0.073
  -0.131 -0.16  -0.08   0.066 -0.23   0.259 -0.163 -0.195  0.029 -0.313]
 [ 0.169 -0.163  0.077 -0.052  0.113  0.031  0.181 -0.216  0.277  0.15
  -0.214 -0.268 -0.024 -0.174 -0.208 -0.14   0.121  0.051 -0.058  0.153
  -0.219  0.244 -0.049 -0.342 -0.184  0.214  0.025  0.126 -0.047  0.074]
 [-0.049 -0.008  0.196  0.35   0.047 -0.439 -0.171  0.007 -0.081 -0.094
   0.076 -0.084 -0.071  0.004 -0.043  0.092 -0.002 -0.19  -0.0